# Ноутбук для демонстрации детектора.

# Core import

In [ ]:
import sys
import random
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import cv2
import torch

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

# Init dirs

In [ ]:
try:
    PROJECT_ROOT = pathlib.Path(os.readlink(f'/proc/{os.environ["JPY_PARENT_PID"]}/cwd'))
except FileNotFoundError:
    __file = %pwd
    PROJECT_ROOT = pathlib.Path(__file).parents[1]

DATA_DIR = PROJECT_ROOT / 'SignDetectorAndClassifier' / 'data'
VIDEO_DIR = DATA_DIR / 'reg_videos'

video_path = str(VIDEO_DIR / '1.mp4')
print(video_path)

In [ ]:
from datetime import datetime

# FIX THIS PATH BY YOURS
MODEL_ARCHIVE_PATH = 'D:\\Downloads\\detector_config_img_size'

from maddrive_adas.sign_det.detector import YoloV5Detector
from maddrive_adas.sign_det.base import AbstractSignDetector

d: AbstractSignDetector = YoloV5Detector(
    config_path=MODEL_ARCHIVE_PATH
)

In [ ]:
from IPython.display import display, Image
import datetime
now = datetime.datetime.now

# initialize video display handles
video = cv2.VideoCapture(video_path)
display_handle1=display(1, display_id=True)
display_handle2=display(2, display_id=True)

while True:
    # store per iter initial time
    t0 = now()
    
    # get video frame and detector results
    _, frame_src = video.read()
    frame_marks = cv2.cvtColor(frame_src.copy(), cv2.COLOR_BGR2RGB)    
    pred = d.detect(frame_marks)
    
    # plot per ROI rectangle
    for i in range(pred.get_roi_count()):
        COORD_ARR, conf = pred.get_abs_roi(i)
        
        frame_marks = cv2.rectangle(frame_marks, (COORD_ARR[0], COORD_ARR[1]), 
                        (COORD_ARR[2], COORD_ARR[3]), 
                        (255, 0, 0), 
                        3)
        
        frame_marks = cv2.putText(frame_marks, str(round(conf, 3)), 
                           (COORD_ARR[2] - 40,COORD_ARR[3]),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.7, (255, 255, 0),
                           5, cv2.LINE_AA
                          )
        
        frame_marks = cv2.putText(frame_marks, str(round(conf, 3)), 
                           (COORD_ARR[2] - 40, COORD_ARR[3]),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.7, (0, 0, 0),
                           2, cv2.LINE_AA
                          )
    # get processing time
    dt = now() - t0
    frame_src = cv2.putText(
        frame_src, 
        'fps:' + str(round(1 / dt.total_seconds() , 2)),
        (0, 60),
        cv2.FONT_HERSHEY_SIMPLEX,
        3, (0, 0, 0),
        3, cv2.LINE_AA
    )
    
    # transform frame for display handle
    _, frame_src = cv2.imencode('.jpeg', frame_src)
    # conver back to BGR for cv2
    frame_marks = cv2.cvtColor(frame_marks, cv2.COLOR_RGB2BGR)
    _, frame_marks = cv2.imencode('.jpeg', frame_marks)
   
    # update display
    display_handle1.update(Image(data=frame_src.tobytes()))
    display_handle2.update(Image(data=frame_marks.tobytes()))